In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

In [3]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [4]:
def select_season (df, season):
    return df[df['season']==season]

In [5]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [6]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [7]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [8]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [9]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [10]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [11]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [12]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

# Loading data

In [13]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [14]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [15]:
n=4
m=4

In [16]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [17]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


## SVM_LINEAR

In [18]:
season_to_be_predicted='2015/2016'
column_to_use_as_prediction=[54,55]
column_to_predict='Goal/No_Goal'

In [19]:
mix=[allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict]

In [20]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [21]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [22]:
#c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [23]:
a['svm_linear'].mean()

0.601955782312925

In [24]:
b['svm_linear'].mean()

0.601955782312925

## SVM_RBF


In [25]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [26]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [27]:
a['svm_rbf'].mean()

0.5593537414965987

In [28]:
b['svm_rbf'].mean()

0.5599489795918368

## CART

In [29]:
dtc = DecisionTreeClassifier()
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

,dtc
0,0.400000
1,0.200000
2,0.200000
3,0.400000
4,0.600000
5,0.200000
6,0.400000
7,0.600000
8,0.600000
9,0.200000


In [30]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

,dtc
0,0.400000
1,0.200000
2,0.200000
3,0.600000
4,0.800000
5,0.200000
6,0.600000
7,0.400000
8,0.600000
9,0.200000


# Adaboost Classifier 

In [31]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

,bagb
0,0.200000
1,0.400000
2,0.400000
3,0.600000
4,0.800000
5,0.400000
6,0.400000
7,0.600000
8,0.600000
9,0.400000


In [32]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

,bagb
0,0.200000
1,0.200000
2,0.200000
3,0.600000
4,0.800000
5,0.400000
6,0.600000
7,0.400000
8,0.400000
9,0.400000


# Gradient Boosting Classifier


In [33]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

,gradb
0,0.600000
1,0.200000
2,0.200000
3,0.400000
4,0.600000
5,0.400000
6,0.400000
7,0.600000
8,0.400000
9,0.400000


In [34]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', gradb,'gradb')

NameError: name 'Italy' is not defined

# Random Forest

In [35]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
Predict_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

NameError: name 'Italy' is not defined

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

In [ ]:
a=select_season(allchampionships[0].df, '2015/2016')

In [ ]:
a.H_ROUND.unique()

In [ ]:
g=30

In [ ]:
Y_test=a[a.H_ROUND>g]['FTR']

In [ ]:
X_test=a[a.H_ROUND>g].iloc[:,[50,51]]

In [ ]:
Y_train=a[a.H_ROUND<=g]['FTR']
X_train=a[a.H_ROUND<=g].iloc[:,[50,51]]

In [ ]:
svm_linear.fit(X_train,Y_train,)

In [ ]:
svm_linear.decision_function(X_test)

# Ridge & Lasso 

## Ridge

In [36]:
from sklearn.linear_model import Ridge, ridge_regression

In [61]:
# using the french league
y = allchampionships[3].df['Goal/No_Goal']
X = allchampionships[3].df.iloc[:,8:-2]

In [39]:
clf = Ridge(alpha=1.0)
clf.fit(X, y) 
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [62]:
x_col = ['rss', 'intercept']
for i in X.columns:
    x_col.append(i)

In [46]:
from sklearn.linear_model import Ridge
def ridge_regression(independent, dependent, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(independent, dependent)
    y_pred = ridgereg.predict(independent)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-y)**2)
    ret = [rss]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)
    return ret

In [68]:
alpha = 1
tmp_ridge = ridge_regression(X, y, alpha)

In [69]:
ridge_df = pd.DataFrame(columns=x_col)
ridge_df.loc[0] = tmp_ridge
pd.DataFrame(ridge_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,0
Aw_Av_Pnt_H,-0.021070
Aw_Av_T_pnt,-0.016801
Aw_AvGR_H,-0.015956
Aw_Av_TGR,-0.012143
H_Av_Pnt_H,-0.006684
H_Av_T_pnt,-0.005652
SUMM_av_HTGS+ATGR,-0.003839
DIFF_av_HP-AP,-0.002935
H_Av_pnt_A,-0.002559
A_TP3,-0.001487


## Lasso 

In [73]:
from sklearn.linear_model import Lasso
def lasso_regression(independent, dependent, alpha):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True)
    lassoreg.fit(independent, dependent)
    y_pred = lassoreg.predict(independent)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-y)**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

In [96]:
alpha = 0.1
tmp_lasso = lasso_regression(X, y, alpha)

In [97]:
lasso_df = pd.DataFrame(columns=x_col)
lasso_df.loc[0] = tmp_lasso
pd.DataFrame(lasso_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,0
A_TP3,-0.000000
Aw_GS_A,0.000000
Aw_GR_H,-0.000000
Aw_GR_A,0.000000
Aw_TGS,-0.000000
Aw_TGR,0.000000
Aw_AvGS_H,-0.000000
Aw_AvGS_A,0.000000
Aw_AvGR_H,-0.000000
Aw_AvGR_A,0.000000


In [92]:
clf = linear_model.Lasso(alpha=0.1)

In [93]:
clf.fit(X,y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [94]:
print(clf.coef_)

[ 0.         -0.         -0.          0.          0.          0.          0.
  0.00018795  0.          0.          0.          0.          0.          0.
  0.         -0.          0.          0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.          0.         -0.          0.
  0.          0.         -0.          0.         -0.          0.          0.
 -0.         -0.         -0.         -0.00039973 -0.          0.         -0.
  0.         -0.          0.          0.          0.          0.        ]


In [98]:
lasso_df = pd.DataFrame(columns=X.columns)
lasso_df.loc[0] = clf.coef_
pd.DataFrame(lasso_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,0
Aw_T_pnt,-0.000400
H_ROUND,0.000000
Aw_GR_H,-0.000000
Aw_GR_A,0.000000
Aw_TGS,0.000000
Aw_TGR,0.000000
Aw_AvGS_H,-0.000000
Aw_AvGS_A,0.000000
Aw_AvGR_H,-0.000000
Aw_AvGR_A,0.000000
